# Creating the Training Data

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm



DATADIR = "C:\\Users\\Josh\Desktop\\Thesis Data\\Extracted"

CATEGORIES = ["Sharp", "Flat","Quaver","Minim","Staccatissimo"]

#CATEGORIES = ["Sharp", "Flat","Minim","8thRest","16thRest","32ndRest","Accent","AltoClef",
#              "BassClef","Beam1","Crochet","Minim","Natural","QuarterRest",
#             "Quaver","Slur","Staccatissimo","Ties","TrebleClef"]


for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        plt.imshow(img_array, cmap='gray')  # graph it
        plt.show()  # display!

        break  # we just want one for now so break
    break  #...and one more!
    
    
    
    
IMG_SIZE = 150

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap='gray')
plt.show()

training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

import random

random.shuffle(training_data)

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

100%|████████████████████████████████████████████████████████████████████████████████| 353/353 [00:07<00:00, 47.58it/s]


1792


# Splitting the Training Data

In [37]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

[[[[ 93]
   [249]
   [254]
   ...
   [ 90]
   [ 88]
   [ 90]]

  [[100]
   [241]
   [254]
   ...
   [252]
   [255]
   [252]]

  [[128]
   [253]
   [251]
   ...
   [253]
   [250]
   [254]]

  ...

  [[255]
   [255]
   [255]
   ...
   [255]
   [255]
   [255]]

  [[253]
   [253]
   [253]
   ...
   [255]
   [255]
   [255]]

  [[ 90]
   [ 90]
   [ 90]
   ...
   [255]
   [255]
   [255]]]]


# Saving the Training Data 

In [38]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)



#  Inserting Values to the Neural Network

In [10]:

import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [63]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D


model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

#model.add(Dense(64))

model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, epochs=1)

model.save('classifier.model')

Epoch 1/1
1792/1792 [==============================] - ETA: 7:46 - loss: 1.6115 - acc: 0.281 - ETA: 7:02 - loss: 5.7851 - acc: 0.234 - ETA: 6:42 - loss: 4.7509 - acc: 0.260 - ETA: 6:30 - loss: 4.0067 - acc: 0.265 - ETA: 6:31 - loss: 3.5351 - acc: 0.281 - ETA: 6:27 - loss: 3.1936 - acc: 0.286 - ETA: 6:19 - loss: 2.9728 - acc: 0.263 - ETA: 6:10 - loss: 2.7698 - acc: 0.304 - ETA: 6:04 - loss: 2.6028 - acc: 0.319 - ETA: 6:00 - loss: 2.4475 - acc: 0.368 - ETA: 5:55 - loss: 2.3142 - acc: 0.392 - ETA: 5:50 - loss: 2.2155 - acc: 0.403 - ETA: 5:44 - loss: 2.1052 - acc: 0.437 - ETA: 5:36 - loss: 1.9937 - acc: 0.462 - ETA: 5:27 - loss: 1.9286 - acc: 0.479 - ETA: 5:19 - loss: 1.8443 - acc: 0.496 - ETA: 5:12 - loss: 1.7824 - acc: 0.501 - ETA: 5:05 - loss: 1.7089 - acc: 0.522 - ETA: 4:56 - loss: 1.6418 - acc: 0.542 - ETA: 4:48 - loss: 1.5891 - acc: 0.557 - ETA: 4:39 - loss: 1.5485 - acc: 0.569 - ETA: 4:31 - loss: 1.5107 - acc: 0.581 - ETA: 4:23 - loss: 1.4807 - acc: 0.589 - ETA: 4:15 - loss: 1.4311 

# Prediction

In [7]:
def prepare(filepath):
    IMG_SIZE = 150  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

In [8]:
import tensorflow as tf
model = tf.keras.models.load_model('classifier.model')
CATEGORIES = ["Sharp", "Flat","Quaver","Minim","Staccatissimo"]
prediction = model.predict([prepare(r'C:\Users\Josh\Desktop\Thesis Data\Extracted\Flat\17.jpg')])  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT

prediction




## HOW TO GET THE PREDICTION ACCURACY :( 


array([[0., 1., 0., 0., 0.]], dtype=float32)

In [11]:
#print("The image is a: " + CATEGORIES[int(prediction[0][0])] + " symbol")

#for i in np.nditer(prediction):
#    if i == 1:
#        print(CATEGORIES[i])

a = np.array(prediction[0], dtype=np.int)

i=0
while i<len(CATEGORIES):
    if a[i] == 1:
        print("The classified image is a: " + CATEGORIES[i])
        i += 1
    else:
        i += 1
    
val_loss, val_acc = model.evaluate(X,y)
print("The value loss of the classified image is: " + str(val_loss) + " While the accuracy is: " + str(val_acc))
    
    
    
    



The classified image is a: Flat
1792/1792 [==============================] - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 57 - ETA: 54 - ETA: 52 - ETA: 49 - ETA: 47 - ETA: 45 - ETA: 42 - ETA: 40 - ETA: 37 - ETA: 35 - ETA: 32 - ETA: 30 - ETA: 28 - ETA: 25 - ETA: 22 - ETA: 20 - ETA: 17 - ETA: 15 - ETA: 12 - ETA: 10 - ETA: 7 - ETA:  - ETA:  - 144s 80ms/step
The value loss of the classified image is: 0.15482254978269339 While the accuracy is: 0.9542410714285714


In [190]:
#from sklearn.metrics import accuracy_score
#acc = prediction
#accuracy_score(X,y)

import Augmentor

p = Augmentor.Pipeline(r"C:\Users\Josh\Desktop\Thesis Data\Extracted\Staccatissimo\Testing")

p.rotate(probability=0.8, max_left_rotation=13, max_right_rotation=13)
p.zoom(probability=0.6, min_factor=1.1, max_factor=1.5)
#p.sample(300)

Initialised with 0 image(s) found.
Output directory set to C:\Users\Josh\Desktop\Thesis Data\Extracted\Staccatissimo\Testing\output.

In [152]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])




ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_1_1/MaxPool' (op: 'MaxPool') with input shapes: [?,1,148,32].